<a href="https://colab.research.google.com/github/Yuanxyyds/JSC270_HW2_2022_Steven_Liu/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>